# Sparse Matrix data structures

We consider the following simple matrix.

In [3]:
import numpy as np

A = np.array([
    [1, 0, 0, 2, 0],
    [3, 4, 0, 5, 0],
    [6, 0, 7, 8, 9],
    [0, 0, 10, 11, 0],
    [0, 0, 0, 0, 12]
])
print(A)

[[ 1  0  0  2  0]
 [ 3  4  0  5  0]
 [ 6  0  7  8  9]
 [ 0  0 10 11  0]
 [ 0  0  0  0 12]]


In the following we want to give a simple overview of sparse data formats to store this matrix. In this example, although we have a number of zero entries, sparse matrix formats are not worthwile, and we use this example mainly for didactical purposes.

## The COO (Coordinate) format

We start with the COO format. It is the most simple format. Let us conver the matrix into it.

In [5]:
from scipy.sparse import coo_matrix

A_coo = coo_matrix(A)

The coo format is a very simple format that explicitly stores the row entries. It consists of three arrays, the row indices, the column indicies and the data entries. Let us print those arrays.

In [6]:
print(A_coo.row)
print(A_coo.col)
print(A_coo.data)

[0 0 1 1 1 2 2 2 2 3 3 4]
[0 3 0 1 3 0 2 3 4 2 3 4]
[ 1  2  3  4  5  6  7  8  9 10 11 12]


We can easily print out the triplets of row index, column index and associated data entry.

In [9]:
list(zip(A_coo.row, A_coo.col, A_coo.data))

[(0, 0, 1),
 (0, 3, 2),
 (1, 0, 3),
 (1, 1, 4),
 (1, 3, 5),
 (2, 0, 6),
 (2, 2, 7),
 (2, 3, 8),
 (2, 4, 9),
 (3, 2, 10),
 (3, 3, 11),
 (4, 4, 12)]

The coo format in Scipy is most frequently used for the generation of sparse matrices. The format is very simple and we can use it to easily create sparse matrices. We only need to provide the row, column and data arrays to create the coo matrix. A major advantage is also that we can repeat indices. In the matrix creation all data entries associated with the same matrix entry is just summed up. This is a very natural operation and simplifies a number of a situations, where we need to create sparse matrices.

However, coo is not a suitable format for typical matrix operations. Also, it is not yet optimal in terms of storage requirements.

## The CSR (Compressed Sparse Row) Format

If we look at the printout of the indices above in the coo format we can see that there is a lot of repetition in the row indices. We store for each nonzero entry the row index even though all row indices within the same row are identical. This motivates the idea of the CSR (Compressed Sparse Row) format. Instead of the row array we store an array of index pointers that give the starting position of the row within the column array. Let us demonstrate how this works.

We first conver the COO matrix format into the CSR format.

In [12]:
A_csr = A_coo.tocsr()

Let us now print out the arrays that define the CSR format. We have three arrays.

* A_csr.data - The data array containing the nonzero entries
* A_csr.indices - The column indices for the nonzero entries
* A_csr.indptr - Pointers into the column indices to store which indices belong to which row.

The first two are the same as in the COO format. The last one requires explanation. For this let us print out the three arrays.

In [13]:
print(A_csr.data)
print(A_csr.indices)
print(A_csr.indptr)

[ 1  2  3  4  5  6  7  8  9 10 11 12]
[0 3 0 1 3 0 2 3 4 2 3 4]
[ 0  2  5  9 11 12]


Comparing the arrays shows that the first two are indeed identical to the corresponding arrays for the COO format. The third array tells us where in the `indices` array the column indices for the ith row are located, namely we have that the column indices for the ith row are located in

```
indices[indptr[i] : indptr[i + 1]]
```
Correspondingly the assocated data entries are in

```
data[indptr[i] : indptr[i + 1]]
```

Let us consider the above example again. The 0th row has column entries starting at index 0. The first row has indices starting at index 2, and so on.